In [2]:
import requests
import pprint
import os

import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd
import numpy as np

pp = pprint.PrettyPrinter()

In [94]:
cid = os.environ.get('SPOTIPY_CLIENT_ID')
secret = os.environ.get('SPOTIPY_CLIENT_SECRET')
username = ""
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
scope = 'user-library-read playlist-read-private'
token = util.prompt_for_user_token(username, scope)
if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

# Get audio features from playlist

In [95]:
df_playlist = pd.DataFrame()
track_list = ''
uploader_list = []
added_ts_list = []
artist_list = []
title_list = []
all_songs = True
offset_index = 0

while all_songs:
    playlist = sp.user_playlist_tracks('jkwd93', playlist_id='3n2aQEZQGEnmavvfOVd3hY', limit=50, offset=offset_index)
    offset_index += 50
    num_songs = len(playlist['items'])
    if num_songs != 50:
        all_songs = False
        
    for song in playlist['items']:
        track_list += song['track']['id'] +','
        
        added_ts_list.append(song['added_at'])
        
        title_list.append(song['track']['name'])
        
        artists = song['track']['artists']
        artists_name = ''
        for artist in artists:
            artists_name += artist['name']  + ','
        artist_list.append(artists_name[:-1])
        
        uploader_list.append(song['added_by']['id'])
        
    track_features = sp.audio_features(track_list[:-1])
    df_temp = pd.DataFrame(track_features)
    df_playlist = df_playlist.append(df_temp)
    track_list = ''

df_playlist['added_at'], df_playlist['song_title'], df_playlist['artists'], df_playlist['uploaded_by']  = added_ts_list, title_list, artist_list, uploader_list

In [102]:
df_playlist.to_hdf('chill_playlist.h5',key='df', mode='w')

In [108]:
df_playlist.head()

,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,...,tempo,time_signature,track_href,type,uri,valence,added_at,song_title,artists,uploaded_by
0,0.897,https://api.spotify.com/v1/audio-analysis/69RS...,0.690,178239,0.263,69RSizl3ZhHgpsI9bVTM51,0.000108,9,0.0803,-12.004,...,118.906,4,https://api.spotify.com/v1/tracks/69RSizl3ZhHg...,audio_features,spotify:track:69RSizl3ZhHgpsI9bVTM51,0.182,2018-04-22T12:08:58Z,Two Bodies - Lido Remix,"Flight Facilities,Emma Louise,Lido",jkwd93
1,0.305,https://api.spotify.com/v1/audio-analysis/4Ovs...,0.613,180950,0.228,4Ovsc4owid3ZTPJ3KeByvr,0.198000,11,0.0815,-14.623,...,129.671,4,https://api.spotify.com/v1/tracks/4Ovsc4owid3Z...,audio_features,spotify:track:4Ovsc4owid3ZTPJ3KeByvr,0.155,2018-04-22T12:09:31Z,Try,SRNO,jkwd93
2,0.516,https://api.spotify.com/v1/audio-analysis/7yjp...,0.867,209313,0.555,7yjpgcDoe13jjiapRyNAfu,0.000002,0,0.0897,-7.025,...,103.961,4,https://api.spotify.com/v1/tracks/7yjpgcDoe13j...,audio_features,spotify:track:7yjpgcDoe13jjiapRyNAfu,0.672,2018-04-22T12:09:38Z,There For Ya,"Lil Cats,Ovcoco",jkwd93
3,0.741,https://api.spotify.com/v1/audio-analysis/21oJ...,0.486,244910,0.419,21oJ1K99GBJrE2GVQGVjA0,0.000066,5,0.0960,-6.415,...,119.065,3,https://api.spotify.com/v1/tracks/21oJ1K99GBJr...,audio_features,spotify:track:21oJ1K99GBJrE2GVQGVjA0,0.227,2018-04-22T12:08:42Z,Frozen,Sabrina Claudio,jkwd93
4,0.425,https://api.spotify.com/v1/audio-analysis/4sCo...,0.571,184999,0.336,4sCo5PwVmXiKeItVY1is4M,0.000752,6,0.1030,-9.977,...,81.194,4,https://api.spotify.com/v1/tracks/4sCo5PwVmXiK...,audio_features,spotify:track:4sCo5PwVmXiKeItVY1is4M,0.612,2018-04-22T12:12:16Z,Give It All up,"SRNO,Gia Koka",jkwd93


# Get audio features from saved tracks

In [55]:
df_saved_tracks = pd.DataFrame()
all_songs = True
track_list = ''
offset_index = 0
added_ts_list = []
artist_list = []
title_list = []
while all_songs:
    saved_tracks = sp.current_user_saved_tracks(limit=20, offset=offset_index)
    offset_index += 20
    num_songs = len(saved_tracks['items'])
    if num_songs != 20:
        all_songs = False
        
    for song in saved_tracks['items']:
        track_list += song['track']['id'] +','
        
        added_ts_list.append(song['added_at'])
        
        title_list.append(song['track']['name'])
        
        artists = song['track']['artists']
        artists_name = ''
        for artist in artists:
            artists_name += artist['name']  + ','
        artist_list.append(artists_name[:-1])
        
    track_features = sp.audio_features(track_list[:-1])
    df_temp = pd.DataFrame(track_features)
    df_saved_tracks = df_saved_tracks.append(df_temp)
    track_list = ''

df_saved_tracks['added_at'], df_saved_tracks['song_title'], df_saved_tracks['artists']  = added_ts_list, title_list, artist_list

In [63]:
df_saved_tracks.to_hdf('saved_tracks.h5',key='df', mode='w')

In [109]:
df_saved_tracks.head()

,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,...,speechiness,tempo,time_signature,track_href,type,uri,valence,added_at,song_title,artists
0,0.920,https://api.spotify.com/v1/audio-analysis/64tG...,0.389,202500,0.314,64tGR2SovXarbawym7l95n,0.000742,7,0.0821,-11.051,...,0.0382,138.076,4,https://api.spotify.com/v1/tracks/64tGR2SovXar...,audio_features,spotify:track:64tGR2SovXarbawym7l95n,0.0688,2018-10-22T15:50:58Z,Starship,The O'My's
1,0.241,https://api.spotify.com/v1/audio-analysis/4as4...,0.839,194426,0.617,4as4XEOR03oGm1STUKl6pa,0.000000,7,0.3000,-4.692,...,0.3310,143.894,4,https://api.spotify.com/v1/tracks/4as4XEOR03oG...,audio_features,spotify:track:4as4XEOR03oGm1STUKl6pa,0.8210,2018-10-22T12:10:09Z,BBIBBI,IU
2,0.289,https://api.spotify.com/v1/audio-analysis/6rfS...,0.650,257519,0.561,6rfSDR4iQbgkhoC2QwDKWu,0.000001,5,0.1420,-6.189,...,0.0252,79.970,4,https://api.spotify.com/v1/tracks/6rfSDR4iQbgk...,audio_features,spotify:track:6rfSDR4iQbgkhoC2QwDKWu,0.6250,2018-10-19T14:26:27Z,Somebody else's guy,BB Valentine
3,0.308,https://api.spotify.com/v1/audio-analysis/0Vhg...,0.696,246880,0.543,0VhgEqMTNZwYL1ARDLLNCX,0.000000,11,0.0939,-6.164,...,0.0489,107.969,4,https://api.spotify.com/v1/tracks/0VhgEqMTNZwY...,audio_features,spotify:track:0VhgEqMTNZwYL1ARDLLNCX,0.4790,2018-10-11T15:33:27Z,Can I Be Him,James Arthur
4,0.569,https://api.spotify.com/v1/audio-analysis/4TWH...,0.628,230400,0.433,4TWHREp4wv0TmewqR6rgRd,0.000008,1,0.1040,-6.808,...,0.0383,88.074,4,https://api.spotify.com/v1/tracks/4TWHREp4wv0T...,audio_features,spotify:track:4TWHREp4wv0TmewqR6rgRd,0.3430,2018-10-06T15:44:39Z,STAY,BLACKPINK
